In [106]:
train_df['Full Bath'].iloc[594]

2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas_profiling

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_df = train_df.drop('id', axis = 1)

test_id = test_df['id']
test_df = test_df.drop('id', axis = 1)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Overall Qual    1350 non-null   int64 
 1   Gr Liv Area     1350 non-null   int64 
 2   Exter Qual      1350 non-null   object
 3   Garage Cars     1350 non-null   int64 
 4   Garage Area     1350 non-null   int64 
 5   Kitchen Qual    1350 non-null   object
 6   Total Bsmt SF   1350 non-null   int64 
 7   1st Flr SF      1350 non-null   int64 
 8   Bsmt Qual       1350 non-null   object
 9   Full Bath       1350 non-null   int64 
 10  Year Built      1350 non-null   int64 
 11  Year Remod/Add  1350 non-null   int64 
 12  Garage Yr Blt   1350 non-null   int64 
 13  target          1350 non-null   int64 
dtypes: int64(11), object(3)
memory usage: 147.8+ KB


In [3]:
def preprocess_data(data, features):
    categorical_data = {'Po' : 1, 'Fa' : 2, 'TA' : 3, 'Gd' : 4, 'Ex' : 5}
    
    for feature in features:
        data[feature] = np.log1p(data[feature])
    
    # 범주화
    data['Exter Qual'] = data['Exter Qual'].map(categorical_data)
    data['Kitchen Qual'] = data['Kitchen Qual'].map(categorical_data)
    data['Bsmt Qual'] = data['Bsmt Qual'].map(categorical_data)
    
    # 리모델링 연차
#     data['update_house'] = data['Year Remod/Add'] - data['Year Built']
    
#     # 최종 품질
#     data['Total_Qual'] = data['Exter Qual'] + data['Kitchen Qual'] + data['Overall Qual']
#     # 2층 파악
#     data['2nd flr SF'] = data['Gr Liv Area'] - data['1st Flr SF']
#     data['2nd flr'] = data['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0)
    
    data['Garage InOut'] = data.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1)
# #     data['Total Bsmt SF'] = 
    data['out_side_qual'] = + data['Exter Qual']
    data['Year Gap Remod'] = data['Year Remod/Add'] - data['Year Built']
    data['Car Area'] = data['Garage Area']/data['Garage Cars']
    data['2nd flr SF'] = data['Gr Liv Area'] - data['1st Flr SF']
    data['2nd flr'] = data['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0)
    data['Total SF'] = data[['Gr Liv Area',"Garage Area", "Total Bsmt SF"]].sum(axis=1)
    data['Sum Qual'] = data[["Exter Qual", "Kitchen Qual", "Overall Qual"]].sum(axis=1)
    data['Garage InOut'] = data.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1)    
    return data


features_train = ['Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Garage Area', 'target']
features_test = ['Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Garage Area']

train_df = train_df.drop(train_df[train_df['Garage Yr Blt'] == 2207].index)
train_df.reset_index(drop = True)

train_df = preprocess_data(train_df, features_train)
test_df = preprocess_data(test_df, features_test)

In [4]:
from pycaret.regression import *
house_reg = setup(data = train_df, train_size = 0.85, target = 'target', 
                  fold_shuffle = True, session_id= 6, use_gpu = False, )

,Description,Value
0,session_id,6
1,Target,target
2,Original Data,"(1349, 22)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,10
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1146, 56)"


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [5]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

add_metric('NMAE', 'NMAE', NMAE)

Name                                                  NMAE
Display Name                                          NMAE
Score Function       <function NMAE at 0x00000159FD42B4C0>
Scorer                                   make_scorer(NMAE)
Target                                                pred
Args                                                    {}
Greater is Better                                     True
Custom                                                True
Name: NMAE, dtype: object

In [6]:
top4 = compare_models(exclude = ['ransac'], round = 6, fold = 10, n_select= 4)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE,TT (Sec)
catboost,CatBoost Regressor,0.097799,0.021697,0.144086,0.856989,0.011294,0.008183,0.008107,0.347000
ridge,Ridge Regression,0.100779,0.022962,0.146975,0.850531,0.011591,0.008441,0.008354,0.003000
br,Bayesian Ridge,0.101125,0.024080,0.149652,0.843580,0.011855,0.008472,0.008383,0.003000
gbr,Gradient Boosting Regressor,0.101364,0.023776,0.151178,0.842515,0.011873,0.008478,0.008403,0.027000
lr,Linear Regression,0.102199,0.025642,0.153133,0.834072,0.012192,0.008565,0.008472,0.214000
rf,Random Forest Regressor,0.103440,0.025534,0.155161,0.833668,0.012171,0.008655,0.008575,0.054000
lightgbm,Light Gradient Boosting Machine,0.107265,0.025242,0.155837,0.832905,0.012186,0.008966,0.008892,0.025000
et,Extra Trees Regressor,0.105146,0.027126,0.159939,0.822475,0.012597,0.008793,0.008717,0.044000
huber,Huber Regressor,0.110285,0.027625,0.161653,0.819785,0.012639,0.009228,0.009143,0.008000
xgboost,Extreme Gradient Boosting,0.110103,0.028839,0.164794,0.810632,0.013014,0.009215,0.009127,0.093000


In [7]:
catboost_reg = create_model('catboost', fold = 10, round = 6)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE
0,0.094706,0.012933,0.113722,0.893120,0.008728,0.007854,0.007836
1,0.091311,0.014816,0.121720,0.896299,0.009310,0.007562,0.007557
2,0.100136,0.023293,0.152620,0.855558,0.011908,0.008385,0.008286
3,0.106067,0.051084,0.226018,0.756739,0.019046,0.009173,0.008799
4,0.088763,0.018909,0.137511,0.875141,0.010714,0.007425,0.007376
5,0.095881,0.017267,0.131404,0.861209,0.010097,0.007954,0.007939
6,0.115489,0.027097,0.164613,0.822355,0.012765,0.009635,0.009556
7,0.088883,0.016159,0.127117,0.860497,0.009862,0.007438,0.007378
8,0.098585,0.017939,0.133938,0.895995,0.010256,0.008171,0.008167
9,0.098167,0.017476,0.132199,0.852979,0.010257,0.008235,0.008180


In [8]:
blender = blend_models(top4, )

,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE
0,0.0879,0.0116,0.1077,0.9042,0.0083,0.0073,0.0073
1,0.0885,0.0142,0.1190,0.9009,0.0091,0.0073,0.0073
2,0.1072,0.0340,0.1844,0.7891,0.0148,0.0090,0.0089
3,0.1063,0.0509,0.2257,0.7575,0.0190,0.0092,0.0088
4,0.0879,0.0182,0.1349,0.8798,0.0105,0.0074,0.0073
5,0.0940,0.0155,0.1245,0.8753,0.0096,0.0078,0.0078
6,0.1142,0.0251,0.1586,0.8352,0.0123,0.0095,0.0095
7,0.0836,0.0143,0.1197,0.8762,0.0093,0.0070,0.0069
8,0.0957,0.0165,0.1285,0.9043,0.0098,0.0079,0.0079
9,0.0931,0.0159,0.1262,0.8660,0.0098,0.0078,0.0078


In [11]:
tmp = tune_model(catboost_reg, n_iter = 100)

,MAE,MSE,RMSE,R2,RMSLE,MAPE,NMAE
0,0.0899,0.0115,0.1071,0.9052,0.0082,0.0075,0.0074
1,0.0894,0.0144,0.1200,0.8992,0.0092,0.0074,0.0074
2,0.1046,0.0247,0.1570,0.8471,0.0121,0.0087,0.0087
3,0.1063,0.0505,0.2246,0.7597,0.0189,0.0092,0.0088
4,0.0907,0.0203,0.1427,0.8656,0.0111,0.0076,0.0075
5,0.0923,0.0158,0.1255,0.8733,0.0097,0.0077,0.0076
6,0.1221,0.0311,0.1763,0.7963,0.0137,0.0102,0.0101
7,0.0817,0.0141,0.1188,0.8781,0.0092,0.0068,0.0068
8,0.1031,0.0196,0.1398,0.8866,0.0107,0.0085,0.0085
9,0.0969,0.0171,0.1310,0.8557,0.0102,0.0081,0.0081


In [23]:
# for i in tmp.get_all_params:
#     print(i)

tmp.learning_rate_

0.029999999329447746

In [1]:
tmp

NameError: name 'tmp' is not defined

In [137]:
tmp_predict = predict_model(tmp, data = test_df)
tmp_predict.head()

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,update_house,Total_Qual,2nd flr SF,2nd flr,Garage InOut,Label
0,9,7.496097,4,2,6.555357,5,7.496097,7.496097,5,2,2007,2007,2007,0,18,0.000000,0,0,12.689828
1,6,6.987490,3,1,5.484797,3,6.987490,6.987490,3,1,1948,1950,1948,2,12,0.000000,0,0,11.781532
2,6,7.361375,4,2,6.089045,4,6.629363,6.646391,4,2,2000,2000,2000,0,14,0.714985,1,1,12.094589
3,6,7.801391,4,3,6.613384,4,7.055313,7.055313,4,2,2004,2004,2004,0,14,0.746078,1,1,12.527191
4,5,6.947937,3,2,6.532334,3,6.947937,6.947937,3,1,1968,1968,1991,0,11,0.000000,0,0,11.799510


In [138]:
def back_target(data):
    y_predict = np.floor(np.expm1(data))
    return y_predict
tmp_predict['Label'] =  back_target(tmp_predict['Label'])

sub_park = pd.DataFrame({
    'id' : test_id,
    'target': tmp_predict['Label']
})

sub_park.to_csv('submission220131(tune_6).csv', index = False)

In [ ]:
1.      # 0.0962	0.0208	0.1410	0.8607	0.0111	0.0081	0.0080
2. 